## Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Load Training DataFrame

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/data (1)/train.csv')
print(train.shape)

In [ ]:
train.head(10)

## Label Distribution

In [ ]:
emotion_prop = (train.emotion.value_counts() / len(train)).to_frame().sort_index(ascending=True)
emotion_prop

In [ ]:
emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

## View Sample of Images

In [ ]:
def pixels_to_array(pixels):
    array = np.array(pixels.split(),'float64')
    return array

def image_reshape(data):
    image = np.reshape(data['pixels'].to_list(),(data.shape[0],48,48,1))
    return image

Conversion

In [ ]:
#CNN
train['pixels'] = train['pixels'].apply(pixels_to_array)
X = image_reshape(train)
y = train['emotion']

In [ ]:
#SVM AND PCA
X_lin = train.drop("emotion",axis=1)
X_lin = np.reshape(train['pixels'].to_list(),(train.shape[0],2304))
new = pd.DataFrame(X_lin)
new

## Split Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)
X_train_lin, X_valid_lin, y_train_lin, y_valid_lin = train_test_split(new, y, test_size=0.3, random_state=1)
X_train_lin = scaler.fit_transform(X_train_lin)
X_valid_lin = scaler.transform(X_valid_lin)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
scaler = StandardScaler()
X_train_lin_pca = pca.fit_transform(X_train_lin)
X_valid_lin_pca = pca.transform(X_valid_lin)
X_train_lin_pca = scaler.fit_transform(X_train_lin)
X_valid_lin_pca = scaler.transform(X_valid_lin)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

In [ ]:
print(X_train_lin.shape)
print(y_train_lin.shape)
print(X_valid_lin.shape)
print(y_valid_lin.shape)

SVM

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state = 1)
classifier.fit(X_train_lin,y_train_lin)

In [ ]:
Y_pred = classifier.predict(X_valid_lin)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_valid_lin,Y_pred)
accuracy = float(cm.diagonal().sum())/len(y_valid_lin)
print("\nAccuracy Of SVM For The Given Dataset : ", accuracy)

In [ ]:
#@title
cm

In [ ]:
# import pickle
# # save the model to disk
filename = 'finalized_model_1.sav'
pickle.dump(classifier, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_valid_lin, y_valid_lin)

In [ ]:
result

SVM+PCA

In [ ]:
from sklearn.svm import SVC
classifier_1 = SVC(kernel='rbf', random_state = 1)
classifier_1.fit(X_train_lin_pca,y_train_lin)

In [ ]:
Y_pred_pca = classifier_1.predict(X_valid_lin_pca)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_valid_lin,Y_pred_pca)
accuracy = float(cm.diagonal().sum())/len(y_valid_lin)
print("\nAccuracy Of SVM+PCA For The Given Dataset : ", accuracy)

In [ ]:
cm

In [ ]:
import pickle
# save the model to disk
filename = 'finalized_model_2.sav'
pickle.dump(classifier_1, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_valid_lin_pca, y_valid_lin)

## Build Network

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

cnn = Sequential([
    Conv2D(64, (3,3), activation = 'relu', padding = 'same', input_shape=(48,48,1)),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    BatchNormalization(),

    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),
    
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    Dense(7, activation='softmax')
])

cnn.summary()

## Train Network

In [ ]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
%%time 

h1 = cnn.fit(
    X_train, y_train, 
    batch_size=256,
    epochs = 20,
    verbose = 1,
    validation_data = (X_valid, y_valid)
)

cnn.summary()

In [ ]:
history = h1.history
print(history.keys())

In [ ]:
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

In [ ]:
%%time 

h2 = cnn.fit(
    X_train, y_train, 
    batch_size=256,
    epochs = 20,
    verbose = 1,
    validation_data = (X_valid, y_valid)
)

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = cnn.predict(X_valid,batch_size=32)
pred_class = np.argmax(y_pred, axis=-1)
print(classification_report(y_valid, pred_class))
print(confusion_matrix(y_valid, pred_class))
